In [ ]:
import pandas as pd

In [ ]:
"""
Using the lookup query to inspect the DB tables
"""

import cdutils.database.connect # type: ignore
from sqlalchemy import text # type: ignore

def fetch_data():
    """
    Main data query
    """
    # Engine 1
    lookup_df = text("""
    SELECT 
        *
    FROM 
        sys.all_tab_columns col
    """)

    persuerfield = text(f"""
    SELECT 
        *
    FROM 
        OSIBANK.PERSUSERFIELD a
    WHERE
        a.USERFIELDCD  = 'PTTM' OR a.USERFIELDCD = 'PTTR'
    """)

    wh_pers = text(f"""
    SELECT 
        *
    FROM 
        OSIBANK.WH_PERS
    """)

    queries = [
        # {'key':'acctcommon', 'sql':acctcommon, 'engine':2},
        {'key':'lookup_df', 'sql':lookup_df, 'engine':1},
        {'key':'PERSUSERFIELD', 'sql':persuerfield, 'engine':1},
        {'key':'WH_PERS', 'sql':wh_pers, 'engine':1}
    ]


    data = cdutils.database.connect.retrieve_data(queries)
    return data


In [ ]:
data = fetch_data()

In [ ]:
wh_pers = data['WH_PERS'].copy()
wh_pers = wh_pers[['persnbr', 'employeeyn']]
wh_pers

In [ ]:
lookup_df = data['lookup_df'].copy()

In [ ]:
lookup_df

In [ ]:
"""
Finding historical data for a given account number.
"""

import cdutils.database.connect # type: ignore
from sqlalchemy import text # type: ignore

def fetch_data_persuserfield():
    """
    Main data query
    """
    # Engine 1
    persuerfield = text(f"""
    SELECT 
        *
    FROM 
        OSIBANK.PERSUSERFIELD a
    WHERE
        a.USERFIELDCD  = 'PTTM' OR a.USERFIELDCD = 'PTTR'
    """)

    queries = [
        {'key':'PERSUSERFIELD', 'sql':persuerfield, 'engine':1},
    ]


    data = cdutils.database.connect.retrieve_data(queries)
    return data


In [ ]:
"""
Finding historical data for a given account number.
"""

import cdutils.database.connect # type: ignore
from sqlalchemy import text # type: ignore

def fetch_data_allroles():
    """
    Main data query
    """
    # Engine 1
    allroles = text(f"""
    SELECT 
        *
    FROM 
        OSIBANK.WH_ALLROLES 
    """)

    queries = [
        {'key':'WH_ALLROLES', 'sql':allroles, 'engine':1},
    ]


    data = cdutils.database.connect.retrieve_data(queries)
    return data


In [ ]:
data = fetch_data_persuserfield()

In [ ]:
persuserfield = data['PERSUSERFIELD'].copy()

get wh_pers and join with persuserfield,

In [ ]:
persuserfield = persuserfield.iloc[:, :3]

In [ ]:
persuserfield = persuserfield.sort_values(by='persnbr', ascending=True)
persuserfield

In [ ]:
pivoted = persuserfield.pivot(index='persnbr', columns='userfieldcd', values='value')

In [ ]:
pivoted

In [ ]:
# fill all missing values with N
pivoted = pivoted.fillna('N')

In [ ]:
pivoted

In [ ]:
only_ppl_with_y = pivoted[(pivoted['PTTM'] == 'Y') | (pivoted['PTTR'] == 'Y')]

In [ ]:
only_ppl_with_y

In [ ]:
roles_data = fetch_data_allroles()

In [ ]:
allroles = roles_data['WH_ALLROLES'].copy()

In [ ]:
allroles

In [ ]:
allroles = allroles[allroles['emplroleyn'] == 'N']
allroles

In [ ]:
df_cleaned = allroles.dropna(subset=['persnbr'])

In [ ]:
df_cleaned

In [ ]:
merged_df = pd.merge(only_ppl_with_y, df_cleaned, on='persnbr')

In [ ]:
merged_df = merged_df.sort_values(by='persnbr', ascending=True)
merged_df

In [ ]:
import pkey
newdata = pkey.pkey()

In [ ]:
newdata['acctnbr'] = newdata['acctnbr'].astype(int)
newdata = newdata[(newdata['curracctstatcd'] == 'ACT') | (newdata['curracctstatcd'] == 'NPFM')]
newdata
# curracctstatcd = act or npfm
# sum total exposure and net bal

In [ ]:
newdata

In [ ]:
merged_df2['acctroledesc'].unique()

In [ ]:
# ['Tax Owner]
# dedupe
# for each persnbr there shouldn't be any duplicate account numbers associated with it
# dont include category ['CRE', 'C&I', 'HOA']

In [ ]:
newdata

In [ ]:
merged_df2 = pd.merge(merged_df, newdata[['acctnbr', 'ownersortname', 'Net Balance', 'Total Exposure', 'Category']], on='acctnbr')

In [ ]:
valid_roles = ['Tax Owner', 'NonTax Owner']
bad_categories = ['CRE', 'C&I', 'HOA']
merged_df2 = merged_df2[merged_df2['acctroledesc'].isin(valid_roles)]
merged_df2 = merged_df2[~merged_df2['Category'].isin(bad_categories)]

In [ ]:
merged_df2

In [ ]:
merged_df2['Category'].unique()

In [ ]:
df_unique = merged_df2.drop_duplicates(subset=['persnbr', 'acctnbr'], keep='first')

In [ ]:
df_unique

In [ ]:
deposits = df_unique[df_unique['Category'].isna()].groupby('persnbr')['Net Balance'].sum()
loans = df_unique[df_unique['Category'].notna()].groupby('persnbr')['Total Exposure'].sum()

summary = pd.concat([deposits, loans], axis=1).fillna(0)

metadata = df_unique.drop_duplicates('persnbr')[['persnbr', 'PTTM', 'PTTR', 'ownersortname']].set_index('persnbr')

final_df = summary.merge(metadata, left_index=True, right_index=True).reset_index()
final_df = final_df[['ownersortname', 'persnbr', 'PTTM', 'PTTR', 'Net Balance', 'Total Exposure']]

In [ ]:
names = {
    'ownersortname': 'Customer Name',
    'persnbr': 'Customer Number',
    'PTTM': 'PTTM',
    'PTTR': 'PTTR',
    'Net Balance': 'Deposit Total Owner Balance',
    'Total Exposure': 'Loans Total Owner Balance'
}

final_df = final_df.rename(columns=names)

In [ ]:
final_df